In [1]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')

tokenizer.save_pretrained('bert-base-chinese')
model.save_pretrained('bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 读取数据
import json

with open('./data/z_medical_stroke_all.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
print(data[0])

# 将数据的instruction替换
for i in range(len(data)):
    data[i]['instruction'] = "现在要做一个根据脑卒中病人的主诉、现病史、既往史和分级表进行严重程度分级的任务。案例会给出主诉、现病史、既往史。\n\t分级如下:\n\t正常\n\t轻度\n\t中度\n\t重度\n\t重度以上\n请根据病人的主诉、现病史、既往史进行分级。\n"

# 现在的数据的output字段是例如 2分 5分, 对其进行转换 0-1:正常 2-4:轻度 5-15:中度 16-20:重度 21-42:重度以上
one = 0
two = 0
three = 0
four = 0
five = 0
for i in range(len(data)):
    if data[i]['output'] == '0分' or data[i]['output'] == '1分':
        data[i]['output'] = "正常"
        one += 1
    elif data[i]['output'] == '2分' or data[i]['output'] == '3分' or data[i]['output'] == '4分':
        data[i]['output'] = "轻度"
        two += 1
    elif data[i]['output'] == '5分' or data[i]['output'] == '6分' or data[i]['output'] == '7分' or data[i]['output'] == '8分' or data[i]['output'] == '9分' or data[i]['output'] == '10分' or data[i]['output'] == '11分' or data[i]['output'] == '12分' or data[i]['output'] == '13分' or data[i]['output'] == '14分' or data[i]['output'] == '15分':
        data[i]['output'] = "中度"
        three += 1
    elif data[i]['output'] == '16分' or data[i]['output'] == '17分' or data[i]['output'] == '18分' or data[i]['output'] == '19分' or data[i]['output'] == '20分':
        data[i]['output'] = "重度"
        four += 1
    elif data[i]['output'] == '21分' or data[i]['output'] == '22分' or data[i]['output'] == '23分' or data[i]['output'] == '24分' or data[i]['output'] == '25分' or data[i]['output'] == '26分' or data[i]['output'] == '27分' or data[i]['output'] == '28分' or data[i]['output'] == '29分' or data[i]['output'] == '30分' or data[i]['output'] == '31分' or data[i]['output'] == '32分' or data[i]['output'] == '33分' or data[i]['output'] == '34分' or data[i]['output'] == '35分' or data[i]['output'] == '36分' or data[i]['output'] == '37分' or data[i]['output'] == '38分' or data[i]['output'] == '39分' or data[i]['output'] == '40分' or data[i]['output'] == '41分' or data[i]['output'] == '42分':
        data[i]['output'] = "重度以上"
        five += 1
print(f"正常:{one} 轻度:{two} 中度:{three} 重度:{four} 重度以上:{five}")



# 将替换后的数据保存为jsonl格式 instuction + input 作为 input字段 output作为output字段
input_len = []
with open('./data/train.jsonl', 'w', encoding='utf-8') as f:
    for i in range(len(data)):
    # for i in range(64):
        f.write(json.dumps({'input': data[i]['instruction'] + data[i]['input'], 'output': data[i]['output']}, ensure_ascii=False) + '\n')
        lens = len(data[i]['instruction'] + data[i]['input'])
        input_len.append(lens)
max_len = max(input_len)
print(f"max_len: {max_len}")

# 正常;轻度;中度各8个作为eval
one = 0; two = 0; three = 0; four = 0; five = 0
eval_data = []
with open('./data/eval.jsonl', 'w', encoding='utf-8') as f:
    for i in range(len(data)):
        if data[i]['output'] == '正常' and one < 8:
            f.write(json.dumps({'input': data[i]['instruction'] + data[i]['input'], 'output': data[i]['output']}, ensure_ascii=False) + '\n')
            one += 1
        elif data[i]['output'] == '轻度' and two < 8:
            f.write(json.dumps({'input': data[i]['instruction'] + data[i]['input'], 'output': data[i]['output']}, ensure_ascii=False) + '\n')
            two += 1
        elif data[i]['output'] == '中度' and three < 8:
            f.write(json.dumps({'input': data[i]['instruction'] + data[i]['input'], 'output': data[i]['output']}, ensure_ascii=False) + '\n')
            three += 1
        else:
            eval_data.append(data[i])
print(f"eval: 正常:{one} 轻度:{two} 中度:{three} 重度:{four} 重度以上:{five}")

In [ ]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
import torch
import os

In [ ]:
config = BertConfig.from_pretrained('./bert-base-chinese')
config.num_labels = 5
config.problem_type = 'multi_label_classification'
tokenizer = BertTokenizer.from_pretrained('./bert-base-chinese')

model = BertForSequenceClassification.from_pretrained('./bert-base-chinese', config=config)

if torch.cuda.is_available() and os.name == 'posix':
    device = torch.compile(model)

In [ ]:
print(tokenizer.decode(tokenizer.encode('你好')))
print(model, model.num_parameters())

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import json

class MyDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_len=512, problem_type='multi_label_classification'):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = json.loads(line)
                self.data.append((line['input'], line['output']))
        self.problem_type = problem_type

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_text, label = self.data[idx]
        input_ids = self.tokenizer.encode(input_text, max_length=self.max_len, padding='max_length', truncation=True, pad_to_max_length=True)
        
        if label == '正常':
            label = 0
        elif label == '轻度':
            label = 1
        elif label == '中度':
            label = 2
        elif label == '重度':
            label = 3
        elif label == '重度以上':
            label = 4
        else:
            raise ValueError("Invalid label")
        
        if self.problem_type == 'single_label_classification':
            return {
                'input_ids': torch.tensor(input_ids, dtype=torch.long),
                'label': torch.tensor(label, dtype=torch.long)
            }
        elif self.problem_type == 'multi_label_classification':
            return {
                'input_ids': torch.tensor(input_ids, dtype=torch.long),
                'label': torch.nn.functional.one_hot(torch.tensor(label), num_classes=5).float()
            }
        else:
            raise ValueError("Invalid problem type")
        
# 构建训练集和验证集
train_dataset = MyDataset('./data/train.jsonl', tokenizer, config.max_position_embeddings, problem_type=config.problem_type)
eval_dataset = MyDataset('./data/eval.jsonl', tokenizer, config.max_position_embeddings, problem_type=config.problem_type)

# 构建DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False)

for i, data in enumerate(train_loader):
    print(data['input_ids'].shape, data['label'].shape)
    break

In [ ]:
# 测试模型在eval上的表现
acc = 0
for i, data in enumerate(eval_loader):
    model.eval()
    input_ids = data['input_ids']
    label = data['label']
    output = model(input_ids)
    output = torch.argmax(output.logits, dim=1)
    label = label.argmax(dim=-1)
    print(f"\noutput:\t{output}\nlabel:\t{label}\n")
    acc += torch.sum(output == label)
print(f"acc: {acc / len(eval_dataset)}")

In [ ]:
# 学习率
import matplotlib.pyplot as plt
def lrate(
    step: int,
    warmup_steps: int = len(train_loader),
    d_model: int = config.hidden_size,
) -> float:
    if step == 0:
        return 0.0
    return (d_model ** -0.5) * min(step ** -0.5, step * warmup_steps ** -1.5)
plt.plot([lrate(step) for step in range(1, len(train_loader) * 10)])

In [ ]:
# 优化器, 学习率调度器, 混合精度训练
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.cuda.amp import GradScaler, autocast

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = AdamW(model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9)
scheduler = LambdaLR(optimizer, lr_lambda=lrate)
scaler = GradScaler(enabled=True) if torch.cuda.is_available() else None

In [ ]:
# 训练函数
import torch.utils.tensorboard as tensorboard
import os
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
# 记录器
writer = tensorboard.SummaryWriter('./logs/medical_stroke')

torch.backends.cudnn.benchmark = True # 加速训练
torch.manual_seed(233)

def trainer(
    model: torch.nn.Module,
    train_loader: DataLoader,
    eval_loader: DataLoader,
    tokenizer: BertTokenizer,
    device: torch.device,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler._LRScheduler,
    scaler: torch.cuda.amp.GradScaler,
    writer: tensorboard.SummaryWriter,
    epochs: int = 10,
):
    # 分布式训练命令 python -m torch.distributed.launch --nproc_per_node=2 train.py
    if torch.cuda.device_count() > 1 and os.name == 'posix':
        print("Using", torch.cuda.device_count(), "GPUs")
        rank = dist.get_rank()
        torch.cuda.set_device(rank % torch.cuda.device_count())
        dist.init_process_group(backend='nccl', init_method='env://')
        model = DDP(model, device_ids=[rank % torch.cuda.device_count()])
        model.to(device)
    else:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = model.to(device)
    global_step = 0
    train_loss = []
    acc = 0 
    ppl = 0

    for epoch in range(epochs):
        model.train()
        for step, batch in enumerate(train_loader):
            input_ids = batch['input_ids'].to(device)
            label = batch['label'].to(device)
            
            # 梯度清零
            optimizer.zero_grad(set_to_none=True)
            # 混合精度训练
            if scaler is not None:
                with autocast():
                    # 前向传播
                    output = model(input_ids, labels=label)
                    loss = output.loss
                # 反向传播
                scaler.scale(loss).backward()
                # 梯度裁剪
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
                # 更新权重
                scaler.step(optimizer)
                scaler.update()
            else:
                output = model(input_ids, labels=label)
                print(output.logits)
                loss = output.loss
                loss.backward()
                optimizer.step()
            # 更新学习率
            scheduler.step()
            # 记录训练损失
            train_loss.append(loss.item())
            global_step += 1

            # 打印训练信息
            if (step+1) % (len(train_loader)//10) == 0:
            # if (step+1) % 1 == 0:
                train_loss = sum(train_loss) / len(train_loss)
                print(f"Epoch [{epoch+1}/{epochs}] Step [{step+1}/{len(train_loader)}] lr: {scheduler.get_last_lr()[0]} loss: {train_loss}")

                # 记录训练信息
                writer.add_scalar('train_lr', scheduler.get_last_lr()[0], global_step)
                writer.add_scalar('train_loss', train_loss, global_step)
                train_loss = []

        # 验证
        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for step, batch in enumerate(eval_loader):
                    input_ids = batch['input_ids'].to(device)
                    label = batch['label'].to(device)
                    output = model(input_ids, labels=label)
                    if config.problem_type == 'single_label_classification':
                        pred = torch.argmax(output.logits, dim=-1)
                        correct += (pred == label).sum().item()
                        total += len(label)
                    elif config.problem_type == 'multi_label_classification':
                        pred = torch.argmax(output.logits, dim=-1)
                        correct += (pred == label.argmax(dim=-1)).sum().item()
                        total += len(label)
                    else:
                        raise ValueError("Invalid problem type")
                acc = correct / total
                print(f"Epoch [{epoch+1}/{epochs}] accuracy: {acc}")
                writer.add_scalar('eval_accuracy', acc, global_step)
    return model

In [ ]:
model = trainer(model, train_loader, eval_loader, tokenizer, device, optimizer, scheduler, scaler, writer, epochs=100)

In [ ]:
# 保存模型
model.save_pretrained('./medical_stroke')